# Setup 

In [2]:
import pandas as pd 
import numpy as np 
import plotly.graph_objects as go 

## Folder Paths 

In [3]:
import os 

folder_path = os.environ.get("NFL_DATA_PATH") 
results_path = os.environ.get("NFL_RESULTS_PATH") 

## Import Play Metadata 

In [4]:
# read in the play metadata
df_meta = pd.read_csv(f"{folder_path}/supplementary_data.csv") 

# showcase the data 
df_meta.head() 

C:\Users\rjfis\AppData\Local\Temp\ipykernel_9976\1083874482.py:2: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df_meta = pd.read_csv(f"{folder_path}/supplementary_data.csv")


,game_id,season,week,game_date,game_time_eastern,home_team_abbr,visitor_team_abbr,play_id,play_description,quarter,...,team_coverage_type,penalty_yards,pre_penalty_yards_gained,yards_gained,expected_points,expected_points_added,pre_snap_home_team_win_probability,pre_snap_visitor_team_win_probability,home_team_win_probability_added,visitor_team_win_probility_added
0,2023090700,2023,1,09/07/2023,20:20:00,KC,DET,3461,(10:46) (Shotgun) J.Goff pass deep left to J.R...,4,...,COVER_2_ZONE,NaN,18,18,-0.664416,2.945847,0.834296,0.165704,-0.081149,0.081149
1,2023090700,2023,1,09/07/2023,20:20:00,KC,DET,461,(7:30) J.Goff pass short right to J.Reynolds t...,1,...,COVER_6_ZONE,NaN,21,21,1.926131,1.345633,0.544618,0.455382,-0.029415,0.029415
2,2023090700,2023,1,09/07/2023,20:20:00,KC,DET,1940,(:09) (Shotgun) J.Goff pass incomplete deep ri...,2,...,COVER_2_ZONE,NaN,0,0,0.281891,-0.081964,0.771994,0.228006,0.000791,-0.000791
3,2023090700,2023,1,09/07/2023,20:20:00,KC,DET,1711,"(:45) (No Huddle, Shotgun) P.Mahomes pass deep...",2,...,COVER_2_ZONE,NaN,26,26,3.452352,2.342947,0.663187,0.336813,0.041843,-0.041843
4,2023090700,2023,1,09/07/2023,20:20:00,KC,DET,1588,(1:54) (Shotgun) P.Mahomes pass incomplete dee...,2,...,COVER_4_ZONE,NaN,0,0,1.921525,-0.324035,0.615035,0.384965,0.000061,-0.000061


# AccelDataPlot 

In [20]:
# class to plot the movement to the ball while it's in the air 
class AccelDataPlot: 

    def __init__(self, game_id, play_id, player_id, df_meta):
        self.game_id = game_id 
        self.play_id = play_id 
        self.player_id = player_id 

        # filter the metadata for the specific play 
        self.play_meta = df_meta.loc[
            (df_meta['game_id'] == game_id) & 
            (df_meta['play_id'] == play_id)
        ].reset_index(drop=True).to_dict(orient='records')[0] 

        # load in the input and output data 
        df_input = pd.read_csv(f"{folder_path}//train//input_2023_w{self.play_meta['week']:02}.csv") 
        df_output = pd.read_csv(f"{folder_path}//train//output_2023_w{self.play_meta['week']:02}.csv") 

        # filter for the given play 
        self.df_input = df_input.loc[
            (df_input["game_id"] == game_id) & 
            (df_input["play_id"] == play_id) &
            (df_input["nfl_id"] == player_id) 
        ].sort_values(["frame_id"]).reset_index(drop = True) 
        self.df_output = df_output.loc[
            (df_output["game_id"] == game_id) & 
            (df_output["play_id"] == play_id) &
            (df_output["nfl_id"] == player_id) 
        ].sort_values(["frame_id"]).reset_index(drop = True) 

        # verify if the player is in the output data 
        if len(self.df_output.index) == 0:
            print(f"Player {player_id} not found in output data for game {game_id}, play {play_id}.")

        # otherwise, continue with the other calculations 
        else:      
            self.calc_relative_distances() 
            self.plot_distance() 
            self.plot_speed() 
            self.plot_acceleration() 
    
    # method to calculate distances relative to start position and ball landing position 
    def calc_relative_distances(self): 

        # get the start position of the player 
        self.start_pos = [
            self.df_output.loc[0, "x"], 
            self.df_output.loc[0, "y"]
        ] 

        # get the ball landing position 
        self.ball_land = [
            self.df_input.loc[0, "ball_land_x"], 
            self.df_input.loc[0, "ball_land_y"]
        ] 

        # calculate the distance from the start position to the ball landing position 
        self.start_to_ball_dist = np.sqrt(
            (self.start_pos[0] - self.ball_land[0])**2 + 
            (self.start_pos[1] - self.ball_land[1])**2
        ) 

        # calculate the distance from the start position to each of the positions 
        self.df_output["dist_from_start"] = np.sqrt(
            (self.df_output["x"] - self.start_pos[0])**2 + 
            (self.df_output["y"] - self.start_pos[1])**2
        ) 

        # calculate the distance to the ball landing position at each frame 
        self.df_output["dist_to_ball_land"] = np.sqrt(
            (self.df_output["x"] - self.ball_land[0])**2 + 
            (self.df_output["y"] - self.ball_land[1])**2
        ) 

        # calculate time in seconds based on frame_id (assuming 10 frames per second) 
        self.df_output["time_sec"] = (self.df_output["frame_id"] - 1) / 10.0 

        # calculate speed and acceleration 
        self.df_output["dbl_last"] = self.df_output["dist_to_ball_land"].shift(1) 
        self.df_output["speed_to_ball"] = (self.df_output["dbl_last"] - self.df_output["dist_to_ball_land"]) * 10.0 
        self.df_output["stb_last"] = self.df_output["speed_to_ball"].shift(1) 
        self.df_output["accel_to_ball"] = (self.df_output["speed_to_ball"] - self.df_output["stb_last"]) * 10.0 
    
    # method to plot distance from ball over time 
    def plot_distance(self): 
        fig = go.Figure()

        # add trace for distance from start 
        fig.add_trace(go.Scatter(
            x=self.df_output["time_sec"], 
            y=self.df_output["dist_to_ball_land"], 
            mode='lines+markers', 
            name='Distance to Ball Landing Position'
        )) 

        # style the figure 
        fig.update_layout(
            title = 'Distance from Start of Throw to Ball Landing Position',
            xaxis_title = 'Time (seconds)',
            yaxis_title = 'Distance to Ball (yards)',
            template='plotly_white'
        ) 

        # show the figure 
        fig.show() 
    
    # method to plot speed to ball over time 
    def plot_speed(self): 
        fig = go.Figure()

        # add trace for speed to ball 
        fig.add_trace(go.Scatter(
            x=self.df_output["time_sec"], 
            y=self.df_output["speed_to_ball"], 
            mode='lines+markers',
            name='Speed to Ball'
        )) 

        # style the figure 
        fig.update_layout(
            title = 'Speed Towards Ball over Time',
            xaxis_title = 'Time (seconds)',
            yaxis_title = 'Speed to Ball (yards/second)',
            template='plotly_white'
        ) 

        # show the figure 
        fig.show() 
    
    # method to plot acceleration to ball over time 
    def plot_acceleration(self): 
        fig = go.Figure()

        # add trace for acceleration to ball 
        fig.add_trace(go.Scatter(
            x=self.df_output["time_sec"], 
            y=self.df_output["accel_to_ball"], 
            mode='lines+markers',
            name='Acceleration to Ball'
        )) 

        # style the figure 
        fig.update_layout(
            title = 'Acceleration Towards Ball over Time',
            xaxis_title = 'Time (seconds)',
            yaxis_title = 'Acceleration to Ball (yards/second²)',
            template='plotly_white'
        ) 

        # show the figure 
        fig.show() 


# test it out 
adp = AccelDataPlot(
    game_id = 2023091007, 
    play_id = 197, 
    player_id = 52535, 
    df_meta = df_meta 
) 

adp.df_output 

,game_id,play_id,nfl_id,frame_id,x,y,dist_from_start,dist_to_ball_land,time_sec,dbl_last,speed_to_ball,stb_last,accel_to_ball
0,2023091007,197,52535,1,64.53,41.08,0.000000,4.613458,0.0,NaN,NaN,NaN,NaN
1,2023091007,197,52535,2,64.20,41.54,0.566127,4.072357,0.1,4.613458,5.411005,NaN,NaN
2,2023091007,197,52535,3,63.82,41.99,1.154210,3.502941,0.2,4.072357,5.694164,5.411005,2.831588
3,2023091007,197,52535,4,63.44,42.45,1.750714,2.936153,0.3,3.502941,5.667876,5.694164,-0.262877
4,2023091007,197,52535,5,63.03,42.90,2.358474,2.361355,0.4,2.936153,5.747983,5.667876,0.801070
5,2023091007,197,52535,6,62.62,43.33,2.951373,1.815544,0.5,2.361355,5.458114,5.747983,-2.898691
6,2023091007,197,52535,7,62.23,43.75,3.524046,1.333604,0.6,1.815544,4.819396,5.458114,-6.387181
7,2023091007,197,52535,8,61.85,44.16,4.082744,0.965402,0.7,1.333604,3.682022,4.819396,-11.373742


In [21]:
# test it out 
adp = AccelDataPlot(
    game_id = 2023091007, 
    play_id = 2359, 
    player_id = 53503,  
    df_meta = df_meta 
) 